In [1]:
import cv2
from paddleocr import PaddleOCR
import threading
import queue

# 创建一个队列用于存放视频帧
frame_queue = queue.Queue()

ocr = PaddleOCR(use_angle_cls=True, lang="ch")

def read_video(filename, frame_queue):
    cap = cv2.VideoCapture(filename)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # 将读取到的帧放入队列中
        frame_queue.put(frame)
    # 当视频读取完毕时，向队列中放入一个特殊值表示结束
    frame_queue.put(None)

def detect_video(frame_queue):
    while True:
        # 从队列中获取一帧，如果没有帧则阻塞等待
        frame = frame_queue.get()
        if frame is None:
            # 遇到特殊值（None）则退出循环
            break
        result = ocr.ocr(frame)
        print(result)

if __name__ == "__main__":
    video_path = r'F:\PythonWork\6.grpc\data/1.mp4'

    t1 = threading.Thread(target=read_video, args=(video_path, frame_queue,))
    t2 = threading.Thread(target=detect_video, args=(frame_queue,))

    t1.start()
    t2.start()

    t1.join()
    t2.join()

ModuleNotFoundError: No module named 'paddleocr'

`frame_queue = queue.Queue(maxsize=10)`  # 这样就设置了队列最多只能存放10个frame  


在这个情况下，当队列已满（即已经有10个frame），试图再放入新帧的操作将会阻塞，直到有其他线程从队列中取出一个frame，腾出空间为止。如果未显式指定 maxsize，则队列会无限制地增长，只要内存允许的话，它可以存储任意多的帧。然而，在实际应用中，尤其是处理视频流时，为了避免内存溢出和保证实时性，通常会根据系统的处理能力和需求设定一个合理的缓冲区大小。


上面提到的两个线程 read_video 和 detect_video 的运行顺序是并行启动的，而不是按照特定顺序执行。在多线程环境下，操作系统会根据调度策略在不同的时间片内切换两个线程，所以它们的实际执行看起来像是交错进行的。  
不过，在这段代码中：  
t1 = Thread(target=read_video, args=(video_path,))  
t2 = Thread(target=detect_video)  
t1.start()  
t2.start()  
t1 用于读取视频帧，而 t2 用于处理这些帧。理论上，t1 应该先于 t2 开始填充队列，以便 t2 能够立即开始从队列中取出数据进行处理。但两个线程各自内部的执行不是有序的，而是并发进行的。  
由于有共享资源（即队列），detect_video 线程会等待队列中有帧可用时才进行处理。这样确保了即使两个线程并行运行，处理逻辑依然能按预期顺序进行——即先读取帧，然后处理帧。  
如果需要确保在 detect_video 线程开始之前 read_video 线程至少已经读取了一帧，则可以通过添加适当的同步机制（例如事件或条件变量）来实现这一需求。然而在这个场景下，通过使用队列作为中介，就已经隐式地实现了这种逻辑上的顺序依赖。
